In [3]:
import tqdm
from loader_findata import finloader
import pandas as pd
class windowed_learning_pipeline:
    def __init__(self, _pth:str, _train_size:int, _dropout_size:int, _win_size:int, _win_train_size:int):
        '''
        С помощью метода get_next() выдаёт следующее скользящее окно для обучения LSTM сети
        Аргументы:
        - pth: путь к 9 csv файлам с рыночными данными
        - test_percentage: процент от данных для тестовой выборки
        - dropout_percentage: процент данных, составляющих отступ между train и test
        - win_percentage: процент данных, которые составляют одно окно
        - win_train_percentage: процент данных для обучения, которые составляют одно окно
        - win_test_percentage: процент данных для тренировки, которые составляют одно окно
        '''

        self.train_size = _train_size
        self.data = finloader(_pth)
        self.length =len(self.data)
        self.dropout_size = _dropout_size
        self.win_size = _win_size
        self.win_train_size = _win_train_size
        self.getted_cnt = 0
        self.dropout_flag = 0

    def get_test(self):
        if not self.dropout_flag:
            return None
        else:
            df_test = pd.DataFrame()
            for i in range(self.getted_cnt, self.length): 
                df_test = pd.concat([df_test, self.data.step()], ignore_index=True)
            return df_test
    
    def gone_dropout(self):
        self.dropout_flag = 1
        for i in range(self.dropout_size):
            self.getted_cnt+=1
            self.data.step()
    
    def get_nxt(self):
        if self.getted_cnt + self.win_size >= self.train_size:
            if not self.dropout_flag:
                self.gone_dropout()
                return(None)
            else:
                return(None)


        df_train = pd.DataFrame()
        df_test = pd.DataFrame()
        for i in tqdm.tqdm(range(0, self.win_size)):
            if i < self.win_train_size:
                df_train = pd.concat([df_train, self.data.step()], ignore_index=True)
            else:
                df_test = pd.concat([df_test, self.data.step()], ignore_index=True)
        
        self.getted_cnt += (self.win_size - self.win_train_size)
        return (df_train, df_test)

In [4]:
lrn_pipe = windowed_learning_pipeline(
    _pth = "C:/Users/Student/UPS2025/UPS2025_FinMat/data/clear_data",
    _train_size = 300000,
    _dropout_size = 10000,
    _win_size = 10000,
    _win_train_size = 9000,
)

In [ ]:
total_wins = 0
while True:
    if lrn_pipe.get_nxt()==None:
        break
    total_wins+=1
print("Wins: ", total_wins)

 18%|█▊        | 1785/10000 [02:33<18:13,  7.51it/s]